In [13]:
import OpenEXR
import Imath
import cv2
import os
import h5py
import numpy as np
from plyfile import PlyData, PlyElement


In [2]:

def save_pointcloud_ply(colors, depths, file_name):
    points = []
    for i in range(depths.shape[0]):
        for j in range(depths.shape[1]):
            if depths[i, j] > 0:  # Only include points with valid depth
                x, y, z = j, i, depths[i, j]
                r, g, b = colors[i, j, 0], colors[i, j, 1], colors[i, j, 2]
                points.append((x, y, z, r, g, b))
    
    dtype = [('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
             ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')]
    vertex_array = np.array(points, dtype=dtype)
    el = PlyElement.describe(vertex_array, 'vertex')
    PlyData([el], text=True).write(file_name)


In [22]:
hdf5_files = [path for path in os.listdir('output') if path.endswith('.hdf5')]
for file_path in hdf5_files:
    file_path = os.path.join('output', file_path)
    with h5py.File(file_path, 'r') as h5_file:
        print(file_path)
        depth_data = h5_file['depth'][:]
        depth_data[depth_data > 5] = 0
        depth_data = (depth_data * 1000)
        color_data = h5_file['colors'][:]

        save_pointcloud_ply(color_data, depth_data, file_path.replace('hdf5', 'ply'))


output/4.hdf5
output/3.hdf5
output/1.hdf5
output/5.hdf5
output/0.hdf5
output/2.hdf5
